<a href="https://colab.research.google.com/github/soline013/Machine-Learning-ML/blob/master/Style-Transfer/Neural_Style_Transfer_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras를 사용한 Neural Style Transfer

Version이 다른 것 같다. 추후 수정해보자.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('user uploaded file "{name}" with length {length} bytes.'.format(name=fn, length=len(uploaded[fn])))


Saving mypic.jpg to mypic.jpg
Saving van.jpg to van.jpg
user uploaded file "mypic.jpg" with length 665711 bytes.
user uploaded file "van.jpg" with length 1629935 bytes.


## Import

In [ ]:
import keras
import time
import numpy as np
from keras.preprocessing.image import load_img, img_to_array, save_img
from keras.applications import vgg19
from scipy.optimize import fmin_l_bfgs_b
from keras import backend as K

In [ ]:
#Image Path.
content_image_path = './mypic.jpg'
style_reference_image_path = './van.jpg'

#Image Size.
width, height = load_img(content_image_path).size
img_height = 400 #Fixed
img_width = int(width * img_height / height)

## VGG19 Model

In [ ]:
def preprocess_image(image_path):
  img = load_img(image_path, target_size=(img_height, img_width)) #target_size is necessary.
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = vgg19.preprocess_input(img)
  return img

def deprocess_image(x):
  #vgg19.preprocess_input에서 생기는 변환을 복원하기 위해 ImageNet의 평균 픽셀값을 더한다.
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68

  #Transfer 'BGR' to 'RGB'
  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [ ]:
#Use K.constant.
content_image = K.constant(preprocess_image(content_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

#K.placeholder to hold the Generated Image.
combination_image = K.placeholder((1, img_height, img_width, 3))

#Three Image -> One Batch.
input_tensor = K.concatenate([content_image,
                              style_reference_image,
                              combination_image], axis=0)

#Makes VGG Network. Input is one batch.
#Load Pre-trained ImageNet Weight.
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet',
                    include_top=False)
print('모델 로드 완료.')

80142336/80134624 [==============================] - 2s 0us/step
모델 로드 완료.


`input_tensor`는 Content, Style, Generated Image를 행으로 쌓는다.

`input_tensor`의 차원은 (3, 400, width, 3)이다.

## Content Loss & Style Loss

In [ ]:
def content_loss(base, combination):
  return K.sum(K.square(combination - base))
  
def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
  gram = K.dot(features, K.transpose(features))
  return gram

def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height * img_width
  return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [ ]:
def total_variation_loss(x):
  a = K.square(
      x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
  b = K.square(
      x[:, :img_height-1, :img_width-1, :] - x[:, img_height-1, 1:, :])
  return K.sum(K.pow(a + b, 1.25))

> 생성된 이미지의 픽셀을 사용하여 계산하는 총 변위 손실 입니다. 이로서 생성된 이미지가 공간적인 연속성을 가지도록 도와주며 픽셀의 격자 무늬가 과도하게 나타나는 것을 막아줍니다. 일종의 규제라고 생각하면 됩니다.

## Total Loss

In [ ]:
# 층 이름과 활성화 텐서를 매핑한 딕셔너리
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# 콘텐츠 손실에 사용할 층
content_layer = 'block5_conv2'
# 스타일 손실에 사용할 층
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# 손실 항목의 가중치 평균에 사용할 가중치
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# 모든 손실 요소를 더해 하나의 스칼라 변수로 손실을 정의합니다
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
content_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(content_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

# 손실에 대한 생성된 이미지의 그래디언트를 구합니다
grads = K.gradients(loss, combination_image)[0] #K.gradients.

# 현재 손실과 그래디언트의 값을 추출하는 케라스 Function 객체입니다
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

RuntimeError: ignored

## Gradient Descent Algorithm

In [ ]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [ ]:
from scipy.optimize import fmin_l_bfgs_b
import time

result_prefix = 'style_transfer_result'
iterations = 20



x = preprocess_image(content_image_path) # 초기 값은 타깃 이미지입니다
x = x.flatten() # scipy.optimize.fmin_l_bfgs_b 함수가 벡터만 처리할 수 있기 때문에 이미지를 펼칩니다.
for i in range(iterations):
    print('반복 횟수:', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    # 뉴럴 스타일 트랜스퍼의 손실을 최소화하기 위해 생성된 이미지에 대해 L-BFGS 최적화를 수행합니다
    print('현재 손실 값:', min_val)
    # 생성된 현재 이미지를 저장합니다
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    save_img(fname, img)
    end_time = time.time()
    print('저장 이미지: ', fname)
    print('%d 번째 반복 완료: %ds' % (i, end_time - start_time))

반복 횟수: 0


NameError: ignored